In [4]:
import numpy as np
from PIL import Image
from scipy.linalg import sqrtm
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim, mean_squared_error as mse
from tensorflow.keras.applications.inception_v3 import preprocess_input
import os

In [5]:
# Function to calculate FID score
def calculate_fid(act1, act2):
    """
    Calculate the Fréchet Inception Distance (FID) score between two sets of activations.
    FID compares the distribution of features between the original and processed images.
    """
    mu1, sigma1 = np.mean(act1, axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = np.mean(act2, axis=0), np.cov(act2, rowvar=False)
    ssdiff = np.sum((mu1 - mu2) ** 2.0)
    covmean = sqrtm(sigma1.dot(sigma2) + np.eye(sigma1.shape[0]) * 1e-6)
    
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

# Function to compute activations using a pre-trained ResNet50 model
def get_activations(images, batch_size=32):
    """
    Compute activations from a pre-trained ResNet50 model for a batch of images.
    The activations are used for calculating FID score between original and processed images.
    """
    model = ResNet50(include_top=False, pooling='avg', input_shape=(224, 224, 3))
    activations = []
    for i in range(0, len(images), batch_size):
        batch = images[i:i + batch_size]
        batch_resized = np.array([preprocess_input(np.array(Image.open(img).resize((224, 224)))) for img in batch])
        activations.append(model.predict(batch_resized))
    return np.concatenate(activations, axis=0)

# Main function to calculate metrics
def calculate_metrics(original_images, processed_images):
    """
    Calculate and return the average PSNR, SSIM, MSE, and FID metrics for the original and processed images.
    """
    if len(original_images) != len(processed_images):
        raise ValueError("Original and processed image arrays must have the same length")
    
    total_psnr, total_ssim, total_mse = 0, 0, 0
    originals = [np.array(Image.open(img).convert("RGB"), dtype=np.float32) / 255.0 for img in original_images]
    processeds = [np.array(Image.open(img).convert("RGB"), dtype=np.float32) / 255.0 for img in processed_images]
    
    for original, processed in zip(originals, processeds):
        total_psnr += psnr(original, processed, data_range=1.0)
        total_ssim += ssim(original, processed, multichannel=True, data_range=1.0, win_size=3)
        total_mse += mse(original, processed)
    
    avg_psnr = total_psnr / len(original_images)
    avg_ssim = total_ssim / len(original_images)
    avg_mse = total_mse / len(original_images)
    
    orig_activations = get_activations(original_images)
    proc_activations = get_activations(processed_images)
    fid = calculate_fid(orig_activations, proc_activations)
    
    return {
        "Average PSNR": avg_psnr,
        "Average SSIM": avg_ssim,
        "Average MSE": avg_mse,
        "FID": fid
    }


In [6]:
if __name__ == "__main__":
    original_images = sorted([os.path.join("Original", fname) for fname in os.listdir("Original")])
    processed_images = sorted([os.path.join("Result", fname) for fname in os.listdir("Result")])

    for orig, proc in zip(original_images, processed_images):
        print(f"Original: {os.path.basename(orig)} | Processed: {os.path.basename(proc)}")

    results = calculate_metrics(original_images, processed_images)
    for metric, value in results.items():
        print(f"{metric}: {value}")

Original: 2592.jpg | Processed: 2592.png
Original: 3.jpg | Processed: 3.png
Original: 390.jpg | Processed: 390.png
Original: 4302.jpg | Processed: 4302.png
Original: 432.jpg | Processed: 432.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Average PSNR: 23.52597840175434
Average SSIM: 0.7746514237761188
Average MSE: 0.005432462173579779
FID: 10.07135767267789
